# Installing Net2Brain

In [ ]:
!pip install -U git+https://github.com/cvai-roig-lab/Net2Brain

# Step 1: Feature Extraction

## Using `FeatureExtractor` with a model from Net2Brain

The FeatureExtractor class provides an interface for extracting features from a given model. When initializing this class, you can customize its behavior by setting various parameters:

- `model` (required): The model from which you want to extract features. Either string in combination with a netset (next parameter), or a variable with a model-type.
- `netset` (optional): The netset (collection of networks) that the model belongs to.
- `layers_to_extract` (optional): A list of layer names or indices from which you want to extract features. Default is None, indicating that all layers preset in the toolbox will be used.
- `device` (optional): The device on which to perform the computations, e.g., 'cuda' for GPU or 'cpu' for CPU. Default is None, which will use the device specified in the global PyTorch settings.
- `transforms` (optional): A list of data preprocessing transforms to apply to the input data before passing it through the model. Default is None, which uses the preset transformations
- `pretrained` (optional): A boolean flag indicating whether to use a pretrained model (if available) or to initialize the model with random weights. Default is True, which means that a pretrained model will be used if possible.

- - -


First we need to a dataset to play around with. For that we will use the dataset by [Micheal F. Bonner (2017)](https://www.pnas.org/doi/full/10.1073/pnas.1618228114), which we can download using the `load_dataset` function

In [4]:
from net2brain.utils.download_datasets import load_dataset
stimuli_path, roi_path = load_dataset("bonner_pnas2017")

### Initating FeatureExtractor


To extract the activations of a pretrained model from a netset, you can use the FeatureExtractor class. First, you need to initialize the class by providing the name of the model and the name of the netset. You can find a suitable model and netset by exploring the taxonomy options available in the Net2Brain toolbox, as shown in the previous notebook "0_Exploring_Net2Brain". For instance, in the following example, we will use AlexNet from the standard netset.

In [ ]:
from net2brain.feature_extraction import FeatureExtractor
fx = FeatureExtractor(model='AlexNet', netset='standard', device='cpu')

The `extract` method computes feature extraction from an image dataset. It takes in the following parameters:

- `dataset_path` (required): The path to the images from which to extract the features. The images must be in JPEG or PNG format.
- `save_format` (optional): The format to save the extracted features in. It can be 'npz', 'pt', or 'dataset'. The default is 'npz', which saves the features in NumPy's .npz format. If 'dataset', the features are saved in the format used by the RSA toolbox's Dataset class.
- `save_path` (optional): The path to save the extracted features to. If None, the folder where the features are saved is named after the current date in the format "{year}{month}{day}{hour}{minute}".
- `layers_to_extract` (optional): A list of layer names or indices from which to extract features. If None, the specified layers will be used.

In [ ]:
from net2brain.feature_extraction import FeatureExtractor

fx = FeatureExtractor(model='AlexNet', netset='standard', device='cpu')
fx.extract(dataset_path=stimuli_path, save_format='npz', save_path='AlexNet_Feat')

__Net2Brain__ chooses by default from which layers of the model to extract the features from. You can inspect which layers are selected by default by calling the `layers_to_extract` attribute:

In [ ]:
fx.layers_to_extract

These are not all the layers that **can** be extracted. If you want to see all the layers that can possibly be extracted you you call `get_all_layers()`.

In [ ]:
fx.get_all_layers()

If you wish to change the layers to be extracted you can add it to the `extract` function like with the parameter 
```
fx.extract(..., layers_to_extract=[your_layers])
```

### Using different datatypes

The `extract()` method allows you to customize the storage of activations with the save_format argument. You have three options for specifying the format:

1. **pt**: Activations are saved as separate tensors for each image.
2. **npz**: Activations are saved as separate arrays for each image.
3. **dataset**: Activations are saved using the Dataset class format from the [RSA toolbox](https://rsatoolbox.readthedocs.io/en/stable/).

In [ ]:
from net2brain.feature_extraction import FeatureExtractor

fx = FeatureExtractor(model='ResNet50', netset='standard', device='cpu')
fts_datasets = fx.extract(dataset_path=stimuli_path, save_format='dataset', save_path='ResNet50_dataset_Feat')


If `dataset` is provided as the output format, the function returns a dictionary of a Dataset class with an entry for each of the layers specified:

In [ ]:
fts_datasets.keys()

Each entry is a `dataset` class:

In [ ]:
fts_datasets['layer1']

 These dataset classes are stored separetely in the output folder. You can load these classes back into your code using the `load_dataset` function from the rsatoolbox dataset like so: 

In [ ]:
from pathlib import Path
from rsatoolbox.data.dataset import load_dataset

filename = Path('ResNet50_dataset_Feat\ResNet50_layer1.hdf5')
layer1_dataset = load_dataset(filename, file_type='hdf5')
display(layer1_dataset)

- - - 

- - -

## Using `FeatureExtractor` with your own DNN

You can also incorporate your own custom model with __Net2Brain__. To do this, supply the `FeatureExtractor` with the following components:

1. Your model
2. Optionally, your custom transform function (if not provided, standard ImageNet transformations will be used)
3. The specific layers you want to extract features from

In [5]:
from torchvision import models
from torchvision import transforms as T

# Define a model
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)  # This one exists in the toolbox as well, it is just supposed to be an example!

# Optional: Define Preprocessing transformations
transforms = T.Compose([
    T.Resize((224, 224)), 
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

# Define extractor (Note: NO NETSET NEEDED HERE)
fx = FeatureExtractor(model=model, transforms=transforms, device='cpu')

# Run extractor
fx.extract(dataset_path=stimuli_path, save_format='npz', save_path='ResNet50_Feat', layers_to_extract=['layer1', 'layer2', 'layer3', 'layer4'])

100%|██████████| 50/50 [00:07<00:00,  6.95it/s]
